In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import japanize_matplotlib
import traceback
import random

# 誤差逆伝搬法の計算式


## 概要

    3層のニューラルネットワーク
    X: (50, 1)
    入力層：1
    中間層：(2, 3)
    出力層：1
    活性化関数：ReLU(出力層は恒等関数)
    損失関数：1/2を掛けて、勾配を求めやすくした疑似平均二乗誤差
    勾配の求め方：誤差逆伝播法
    最適化手法：勾配降下法

<img src="image.png"  height="300" width="400"/>


### 損失関数

$$
\mathrm{MSE} = \frac{1}{n} \sum_{i=1}^{n} \frac{1}{2} (y_i - \hat{y}_i)^2
$$

y_pred と y を入れ替え

$$
= \frac{1}{n} \sum_{i=1}^{n} \frac{1}{2} (\hat{y}_i - y_i)^2
$$

ベクトル表記

$$
= \frac{1}{n} \frac{1}{2} \left\| \hat{\mathbf{y}} - \mathbf{y} \right\|^2
$$

In [ ]:
# 実際の値（正解ラベル）
y = np.array([10, 2.0, 3.0])

# 予測値
y_hat = np.array([5, 4, 2])

# サンプル数
n = len(y)

# ベクトル表記の平均二乗誤差（0.5付き）
mse = (1 / (2 * n)) * np.linalg.norm(y_hat - y) ** 2

print("平均二乗誤差（0.5付き）:", mse)

## 誤差逆伝搬法の計算

### Step1. 損失関数を求める

まずは、順伝搬で予測値を求める。

\begin{align*}
U1 &= X W1 + b1        && \quad \text{(50, 2)} \\
Z1 &= \mathrm{ReLU}(U1) && \quad \text{(50, 2)} \\[1em]
U2 &= Z1 W2 + b2      && \quad \text{(50, 3)} \\
Z2 &= \mathrm{ReLU}(U2) && \quad \text{(50, 3)} \\[1em]
U3 &= Z2 W3 + b3      && \quad \text{(50, 1)} \\
Z3 &= Identity(U3)                && \quad \text{(50, 1)} \\
\end{align*}

$$
L = \frac{1}{n} \frac{1}{2} \left\| \mathbf{Z_3} - \mathbf{y} \right\|^2  \quad \text{※Lはスカラ} 
$$


### Step2. Z3の勾配を求める

損失関数をZ3で偏微分する
\begin{align*}
\frac{\partial L}{\partial Z3} 
&= \frac{\partial L}{\partial L} \cdot \frac{\partial L}{\partial Z3} \\[1em]
&= \frac{1}{n} \left( \mathbf{Z3} - \mathbf{y} \right) \\[1em]
&=
\begin{bmatrix}
\frac{\partial L}{\partial Z3_1} \\
\frac{\partial L}{\partial Z3_2} \\
\frac{\partial L}{\partial Z3_3} \\
\vdots \\
\frac{\partial L}{\partial Z3_N}
\end{bmatrix}　\quad \text{(50, 1)}
\end{align*}



### Step3. U3の勾配を求める

損失関数をU3で偏微分する

\begin{align*}
L &= \frac{1}{n} \cdot \frac{1}{2} \left\| \mathbf{Z3} - \mathbf{y} \right\|^2 && \quad \text{※ $L$ はスカラー} \\
\mathbf{Z3} &= \mathrm{Identity}(\mathbf{U3}) \\[1em]

\frac{\partial L}{\partial U3} 
&=\frac{\partial L}{\partial Z3} \frac{\partial Z3}{\partial U3}  \\[1em]
&= \frac{1}{n} \left( \mathbf{Z3} - \mathbf{y} \right) * 1 \quad \text{(50, 1)} \\[1em]


&= \begin{bmatrix}
       \frac{\partial L}{\partial U3_1} \\
       \frac{\partial L}{\partial U3_2} \\
       \frac{\partial L}{\partial U3_3} \\
       \vdots
       \frac{\partial L}{\partial U3_N}

\end{bmatrix}　\quad \text{(50, 1)}
\end{align*}


### Step4. 重みW3の勾配を求める

損失関数をW3で偏微分する

\begin{align*}
L &= \frac{1}{n} \cdot \frac{1}{2} \left\| \mathbf{Z3} - \mathbf{y} \right\|^2 && \quad \text{※ $L$ はスカラー} \\

\mathbf{Z3} &= \mathrm{Identity}(\mathbf{U3}) \\

\mathbf{U3} &= \mathbf{Z2} \cdot \mathbf{W3} + \mathbf{b3} \\[1em]

\frac{\partial L}{\partial W3} 
&= \frac{\partial L}{\partial Z3} \frac{\partial Z3}{\partial U3} \frac{\partial U3}{\partial W3} \\[1em]
&= \mathbf{Z2}^{\mathsf{T}} \cdot  \frac{\partial L}{\partial U3}  \\[1em]

&= \begin{bmatrix}
       \frac{\partial L}{\partial W3_1} \\
       \frac{\partial L}{\partial W3_2} \\
       \frac{\partial L}{\partial W3_3} \\
       \vdots
       \frac{\partial L}{\partial W3_N}

\end{bmatrix}　\quad \text{(3, 1)}
\end{align*}


### Step5. バイアスb3の勾配を求める

損失関数をb3で偏微分する

\begin{align*}
L &= \frac{1}{n} \cdot \frac{1}{2} \left\| \mathbf{Z3} - \mathbf{y} \right\|^2 && \quad \text{※ $L$ はスカラー} \\

\mathbf{Z3} &= \mathrm{Identity}(\mathbf{U3}) \\

\mathbf{U3} &= \mathbf{Z2} \cdot \mathbf{W3} + \mathbf{b3} \\[1em]

\frac{\partial L}{\partial b3} 
&= \frac{\partial L}{\partial Z3} \frac{\partial Z3}{\partial U3} \frac{\partial U3}{\partial b3} \\[1em]

&= \sum_{i=1}^{n} \left( \frac{\partial L}{\partial \mathbf{U3}} \right)_i \quad \in \mathbb{R}^{1 \times 3} \\[1em]

&= \begin{bmatrix}
       \frac{\partial L}{\partial b3_1} \\
       \frac{\partial L}{\partial b3_2} \\
       \frac{\partial L}{\partial b3_3} \\
       \vdots
       \frac{\partial L}{\partial b3_N}

\end{bmatrix}　\quad \text{(1, 1)}
\end{align*}


### Step6. Z2の勾配を求める

損失関数をZ2で偏微分する

\begin{align*}
L &= \frac{1}{n} \cdot \frac{1}{2} \left\| \mathbf{Z3} - \mathbf{y} \right\|^2 && \quad \text{※ $L$ はスカラー} \\[1em]

\mathbf{Z3} &= \mathrm{Identity}(\mathbf{U3}) \\[1em]

\mathbf{U3} &= \mathbf{Z2} \cdot \mathbf{W3} + \mathbf{b3} \\[1em]


\frac{\partial L}{\partial b3} 
&= \frac{\partial L}{\partial Z3} \frac{\partial Z3}{\partial U3} \frac{\partial U3}{\partial Z2} \\[1em]

&=  \frac{\partial L}{\partial U3} \cdot \mathbf{W3}^{\mathsf{T}} \\[1em]

&= \begin{bmatrix}
       \frac{\partial L}{\partial Z2_1} \\
       \frac{\partial L}{\partial Z2_2} \\
       \frac{\partial L}{\partial Z2_3} \\
       \vdots
       \frac{\partial L}{\partial Z2_N}

\end{bmatrix}　\quad \text{(50, 3)}
\end{align*}


### Step6. U2の勾配を求める

損失関数をZ2で偏微分する

\begin{align*}
L &= \frac{1}{n} \cdot \frac{1}{2} \left\| \mathbf{Z3} - \mathbf{y} \right\|^2 && \quad \text{※ $L$ はスカラー} \\[1em]

\mathbf{Z3} &= \mathrm{Identity}(\mathbf{U3}) \\[1em]

\mathbf{U3} &= \mathbf{Z2} \cdot \mathbf{W3} + \mathbf{b3} \\[1em]

\mathbf{Z3} &= \mathrm{ReLU}(\mathbf{U2}) \\[1em]

\frac{\partial L}{\partial b3} 
&= \frac{\partial L}{\partial Z3} \frac{\partial Z3}{\partial U3} \frac{\partial U3}{\partial Z2} \frac{\partial Z2}{\partial U2} \\[1em]

&=  \frac{\partial L}{\partial Z3} ReLU'(U3) \\[1em]

&= \begin{bmatrix}
       \frac{\partial L}{\partial U2_1} \\
       \frac{\partial L}{\partial U2_2} \\
       \frac{\partial L}{\partial U2_3} \\
       \vdots
       \frac{\partial L}{\partial U2_N}

\end{bmatrix}　\quad \text{(50, 3)} \\[2em]


&\text{ReLUの微分は、引数として与えられた}~U~(\ell\text{層の出力})~\text{に対して以下のように適用される：} \\
&\quad U\ell_i > 0 \quad \Rightarrow \quad \frac{\partial L}{\partial Z3_i} \times 1 \\
&\quad U\ell_i \leq 0 \quad \Rightarrow \quad \frac{\partial L}{\partial Z3_i} \times 0 \\
\end{align*}


### Step7. W2の勾配を求める。

... 以下略